In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, fsolve
from scipy.stats import chi2

In [2]:
def chi2_bereken(param, x_val, y_val, y_err, soort_fout, model):
    """Geeft chi^2 waarde in functie van de parameters
    
    Args: 
        param: Waardes voor de parameters van het model
        model: Het gebruikte model dat gefit wordt
        x_val: Een vector van invoerwaardes voor het model
        y_val: Een vector met meetwaardes die gefit moeten worden
        y_err: Een vector met de fouten op y
        
    Return:
        chi_2_val: De chi^2 waarde van het model gegeven de waardes voor de parameters uit param.
        
    """
    if soort_fout == "Unif":
        fouten = y_err**2 / 12
    else:
        fouten = y_err**2
    chi_2_val = np.sum((y_val - model(x_val, param))**2 / fouten)
    return chi_2_val


def minimize_chi2(model, initial_vals, x_val, y_val, y_err, soort_fout = 'Stat'):
    """Minimaliseert de chi^2 waarde voor een gegeven model en een aantal datapunten
    
    Args:
        model: Het gebruikte model dat gefit wordt.
        x_val: Een vector van invoerwaardes voor het model
        y_val: Een vector met meetwaardes die gefit moeten worden
        y_err: Een vector met de fouten op y
        soort_fout: Laat toe om het type fout mee te geven, dit is enkel van belang als de fout op de meetpunten uniform is.
                    
    Return:
        oplossing: Een array dat de minimale waardes voor de parameters geeft
    """
    chi2_func = lambda *args: chi2_bereken(*args)
    gok = initial_vals(x_val, y_val)
    mini = minimize(chi2_func, gok, args = (x_val, y_val, y_err, soort_fout, model))
    return mini

In [3]:
def chi2_in_1_var(var, ind_var, x_val, y_val, y_err, param_values, chi_min, model, soort_fout = "Stat"):
    outp = np.array([])
    aant_param = len(param_values)
    for val in var:
        kopie = param_values.copy()
        np.put(kopie, ind_var, val)
        outp = np.append(outp, chi2_bereken(kopie, x_val, y_val, y_err, soort_fout, model) - chi2.ppf(0.68, df=aant_param) - chi_min)
    return outp

def find_sigma_values(x_val, y_val, y_err, param_values, te_checken_param_ind, chi_min, soort_fout, model):
    functie = lambda *args: chi2_in_1_var(*args)
    gok = param_values[te_checken_param_ind]
    oplossing_max = fsolve(functie, args = (te_checken_param_ind, x_val, y_val, y_err, param_values, chi_min, model, soort_fout), x0 = gok + abs(gok)/2)
    oplossing_min = fsolve(functie, args = (te_checken_param_ind, x_val, y_val, y_err, param_values, chi_min, model, soort_fout), x0 = gok - abs(gok)/2)
    return [oplossing_min[0], oplossing_max[0]]
    
def uncertainty_intervals(min_values, x_val, y_val, y_err,  chi_min, model, soort_fout = "Stat"):
    aant_param = len(min_values)
    intervallen = []
    for i in range(0, aant_param):
        intervallen.append(find_sigma_values(x_val, y_val, y_err, min_values, i, chi_min, soort_fout, model))
    return intervallen

In [4]:
# def opt_fit_and_bare(x_val, y_val, y_err, min_param, x_as_titel, y_as_titel, fig_titel):
#     fig, ax = plt.subplots(1,2, figsize = (12,6))
#     De data alleen plotten
#     ax[0].errorbar(positie, intensiteit, yerr = fouten, fmt = 'ko', ecolor = 'k', capsize = 2.5, markersize = 4, elinewidth = 1)
#     ax[0].set_xlabel(x_as_titel)
#     ax[0].set_ylabel(y_as_titel)
#     ax[0].set_title(fig_titel)
    
#     De data met de optimale fit plotten
#     X = np.linspace(positie[0] - 2, positie[len(positie) - 1], 10000) #Dit is input voor het model
#     ax[1].errorbar(positie, intensiteit, yerr = fouten, fmt = 'ko', ecolor = 'k', capsize = 2, 
#             markersize = 3, elinewidth = 0.85, label = "Meetpunten")
#     ax[1].plot(X, model(X, min_param), 'r--', label = "Optimale fit")
#     ax[1].legend()
#     ax[1].set_xlabel("Positie $[mm]$")
#     ax[1].set_ylabel("Intensiteit $[W/(mm)^2]$")
#     ax[1].set_title("Gemeten intensiteit vs positie")
#     plt.show()
    
# def pretty_print_param(min_param, betr_intervallen, chi_min, x_val, y_val, y_err, parameters):
#     De waardes voor de parameters printen
#     for i in range(0, len(betr_intervallen)):
#         top = betr_intervallen[i][1] - min_param[i]
#         outp = parameters[i] + " heeft als waarde: %.5g met 68%% betrouwbaarheidsinterval: [%.5g, %.5g] "%(min_param[i], betr_intervallen[i][0], betr_intervallen[i][1])
#         print(outp)
        
#     De chi^2 plots maken
#     aant_param = len(betr_intervallen)
#     chi_exp = np.full(1000,chi_min + chi2.ppf(0.68, df=aant_param))
#     titels = ["$\chi^2$ voor $x_0$ rond de best fittende waarde.",
#           "$\chi^2$ voor $\gamma$ rond de best fittende waarde.",
#           "$\chi^2$ voor $A$ rond de best fittende waarde.",
#           "$\chi^2$ voor $y_0$ rond de best fittende waarde."]
#     xlabels = ["$x_0 [mm]$","$\gamma [mm]$", "$A [W/mm]$", "$y_0 [W/(mm)^2]$"]
#     chi_vali_labels = ["$\chi^2$ voor $x_0$ rond de best fittende waarde.",
#                    "$\chi^2$ voor $\gamma$ rond de best fittende waarde.",
#                    "$\chi^2$ voor $A$ rond de best fittende waarde.",
#                    "$\chi^2$ voor $y_0$ rond de best fittende waarde."]
#     fig, ax = plt.subplots(2, 2, figsize = (10, 10))
#     for i in range(0, 2):
#         for j in range(0,2):
#             top en bot zijn respectievelijk de 1 \sigma afwijkingen naar boven en onder.
#             top = betr_intervallen[2*i + j][1] - min_param[2*i + j]
#             bot = min_param[2*i + j] - betr_intervallen[2*i + j][0]
#             i_range = np.linspace(min_param[2*i + j] - bot*1.25, min_param[2*i + j] + top*1.25, 1000)
#             chi_vali = chi2_in_1_var(i_range, 2*i + j, x_val, y_val, y_err, min_param, chi_min) + np.full(1000, chi2.ppf(0.68, df=aant_param) + chi_min)
#             ax[i][j].plot(i_range, chi_vali, label = chi_vali_labels[2*i+j])
#             ax[i][j].plot(i_range, chi_exp, 'k--', label = "$\chi^2 = \chi^2_{min} +$ chi2.ppf(0.68, df = %d)"%aant_param)
#             ax[i][j].plot(betr_intervallen[2*i + j], np.full(2,chi_min + chi2.ppf(0.68, df=aant_param)),'ro', markersize = 5)
#             ax[i][j].set_title(titels[2*i + j])
#             ax[i][j].set_xlabel(xlabels[2*i + j])
#             ax[i][j].set_ylabel("$\chi^2$")
#             ax[i][j].legend()
#     plt.show()
    
# def print_p_val_en_chi2_red(chi_min, x_val, min_param):
#     nu = len(x_val) - len(min_param)
#     p_waarde = chi2.sf(chi_min, df=nu)
#     chi_red = chi_min/nu
#     print("De p-waarde voor de hypothese test dat het model zinvol is, wordt gegeven door: %.5g"%p_waarde)
#     print("De gereduceerde chi^2 waarde is: %.5g"%chi_red)

# def pretty_print_results(x_val, y_val, y_err, chi_min, min_param, betr_intervallen, parameters):
#     opt_fit_and_bare(x_val, y_val, y_err, min_param)
#     pretty_print_param(min_param, betr_intervallen, chi_min, x_val, y_val, y_err)
#     print_p_val_en_chi2_red(chi_min, x_val, min_param)


# pretty_print_results(positie, intensiteit, fouten, chi_min, min_param, betr_intervallen)

In [2]:
def fit(parameters, model, initial_vals, x_val, y_val, y_err, soort_fout = "Stat", 
        x_as_titels = "Generic", y_as_titels = "Generic", titel = "Generic", printen = "False"): #Veel van deze inputs doen niets, kmoet nog pretty
    #print code schrijven
    #TODO: cas_matrix support maken
    #TODO: ML code schrijven
    print("Raw output")
    mini = minimize_chi2(model, initial_vals, x_val, y_val, y_err, soort_fout)
    chi_min = mini["fun"]
    min_param = mini["x"]
    print(mini)
    
    betrouwb_int = uncertainty_intervals(min_param, x_val, y_val, y_err, chi_min, model, soort_fout)
    print(betrouwb_int)
    foutjes = []
    for i in range(0, len(parameters)):
        top = betrouwb_int[i][1] - min_param[i]
        bot = min_param[i] - betrouwb_int[i][0]
        foutjes.append((bot, top))
        outp = parameters[i] + " heeft als waarde: %.5g + %.5g - %.5g met 68%% betrouwbaarheidsinterval: [%.5g, %.5g] "%(min_param[i], top, bot, betrouwb_int[i][0], betrouwb_int[i][1])
        print(outp)

    nu = len(x_val) - len(parameters)
    p_waarde = chi2.sf(chi_min, df=nu)
    chi_red = chi_min/nu
    print("De p-waarde voor de hypothese test dat het model zinvol is, wordt gegeven door: %.5g"%p_waarde)
    print("De gereduceerde chi^2 waarde is: %.5g"%chi_red)
    fouten = []
    for fout in foutjes:
        if fout[0]/fout[1] < 1.25 and fout[0]/fout[1] > 0.8:
            fouten.append(max(fout[0], fout[1]))
        else:
            fouten.append(fout)
    outp = []
    for i in range(0, len(parameters)):
        outp.append([min_param[i], fouten[i], 'S'])
    return outp
    #Werkt nog niet, kmoet de code nog algemeen schrijven :(
    #if printen:
    #    print("##################### Pretty print #####################")
    #    pretty_print_results(x_val, y_val, y_err, chi_min, min_param, betrouwb_int, parameters)
    